## Outline of the notebook

## Part 1: Job Scraping
## Part 2: Using Kaggle Data set to create a list of algorithms, tools and languages used 
## Part 3: Basic Analysis using the Lists
## Part 4: Using TFID vectorizer to create a list of algorithms, tools and languages from Indeed Job postings obtained from part 1
## Part 5: Methodolgy followed for Question 1

## Part 1: Job Scraping, Importing the Job postings from Indeed

Importing the necessary files

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import nltk
import string
import re
import csv
import re
from itertools import zip_longest

import matplotlib.pyplot as plt
from collections import Counter


q2_roles is answering the question 2 of the assignment. We have obtained jobs separetely for both questions 2 and 3

Now we found out which job roles correspond to the Technical Program, We begin by creating a list of technical roles and save it as q2_roles

In [2]:
q2_roles = ['Data+Analyst','Machine+Learning+Engineer','Data+Scientist','Predictive+Modeler','Data+Miner','Database+Engineer','DBA+Engineer', 'Database+Enigneer','Data+Analytics','Data+Architect']

The following are roles corresponding to the Management Program

In [3]:
q3_roles = ['Analytical+Consultant', 'Business+Analyst','Business+Intelligence',
            'Business Integration','Retail+E-Commerce+Coordinator','Retail+Analytics',
            'Forecasting+Analytics+Consultant','Analytics+Strategist']


By seperately scraping both roles we get two different job csv files 

We create a set of cities we believe have the most jobs in data science

In [4]:
#ca_city_set = ['Toronto','Vancouver']
ca_city_set = ['Toronto','Vancouver']
#us_city_set=['New+york','San+francisco','Chicago','Seattle','Redmond','Boston','Raleigh','San+Diego','Dallas','Portland']
us_city_set=['San+francisco','New+york','San+francisco','Chicago','Seattle','Redmond','Boston','Raleigh','San+Diego','Dallas']

Create a function that would extract a job summary from a partifular job and another function that would scrap various 
information about the job and call the first function

Dont run the following cell use the csv files to find our jobs descriptions

In [1]:
def extract_job_summary(soup):
    """Function to extract job summary from soup and save it into a list jobs. """
    jobs = []
    for span in soup.find_all(name = 'span',attrs={'class':'summary'}):
        jobs.append(span.text)
    return(jobs)

def extract_job_info(ca_city_set = ['Toronto'],us_city_set = ['New+york'],max_result = 1,roles = ['Data+Scientist']):
    """Function to extract job info from prettified soup and save it into a dataframe. It takes Canadian city set, US city
    set and max_results. The max_result shows how many pages it should scan through. the site has 15 jobs/page so if max_result
    is equal to 5 then it will return 15 x 5 results per city"""
    
    columns = ['city', 'job_title', 'company_name', 'location','salary','Summaries']
    #create dataframes the info on the jobs from the two countries will be saved
    sample_df = pd.DataFrame(columns = columns)
    sample_df2 = pd.DataFrame(columns = columns)
    #create the loop through the canadian city list
    for city in ca_city_set:
        for job in roles:
            for start in range (0,max_result,10):
                #build the url
                url = 'https://ca.indeed.com/jobs?q='+str(job)+'&l=' + str(city)+'&start='+str(start)
                r = requests.get(url)
                html_doc = r.text
                #use beautifulsoup to parse the html code
                soup = BeautifulSoup(html_doc,'html.parser')
                for div in soup.find_all(name = 'div',attrs= {'class':'row'} ):
                    num = len(sample_df)+1
                    job_post = []
                    #add city
                    job_post.append(city)
                    #add title
                    for a in div.find_all(name = 'a',attrs = {'data-tn-element':'jobTitle'}):
                        job_post.append(a['title'])
                    #add company name
                    company = div.find_all(name = 'span',attrs = {'class':'company'})
                    if len(company) > 0:
                        for b in company:
                            job_post.append(b.text.strip())
                    else:
                        sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                        for span in sec_try:
                            job_post.append(span.text.strip())
                        #add locationo name
                    location = div.find_all(name = 'span',attrs = {'class':'location'})
                    for c in location:
                        job_post.append(c.text.strip())
                    #add salary
                    try:
                        job_post.append(div.find('nobr').text)
                    except:
                        try:
                            try_2 = div.find(name = 'div',attrs = {'class':'sjcl'})
                            try_3 = try_2.find('div')
                            job_post.append(try__3.text.strip())
                        except:
                            job_post.append('No_salary')
                    #add job summary
                    base_url = 'http://www.indeed.com'
                    for a in div.find_all(name = 'a',attrs = {'data-tn-element':'jobTitle'}):
                        c = base_url + str(a['href'])
                        r = requests.get(c)
                        html_doc = r.text
                        soup = BeautifulSoup(html_doc,'html.parser')
                        summary = extract_job_summary(soup)
                        job_post.append(summary)
                    sample_df.loc[num] = job_post
    #loop through the USA city set           
    for job in roles:
        for city in us_city_set:
        
            for start in range (0,max_result,10):
                url = 'https://www.indeed.com/jobs?q='+str(job)+'&l='+str(city) + '&start=' + str(start)
                r = requests.get(url)
                html_doc = r.text
                soup = BeautifulSoup(html_doc,'html.parser')
                for div in soup.find_all(name = 'div',attrs= {'class':'row'} ):
                    num = len(sample_df2)+1
                    job_post = []
                    #add city
                    job_post.append(city)
                    #add title
                    for a in div.find_all(name = 'a',attrs = {'data-tn-element':'jobTitle'}):
                        job_post.append(a['title'])
                    #add company name
                    company = div.find_all(name = 'span',attrs = {'class':'company'})
                    if len(company) > 0:
                        for b in company:
                            job_post.append(b.text.strip())
                    else:
                        sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                        for span in sec_try:
                            job_post.append(span.text.strip())
                        #add locationo name
                    location = div.find_all(name = 'span',attrs = {'class':'location'})
                    for c in location:
                        job_post.append(c.text.strip())
                    #add salary
                    try:
                        job_post.append(div.find('nobr').text)
                    except:
                        try:
                            try_2 = div.find(name = 'div',attrs = {'class':'sjcl'})
                            try_3 = try_2.find('div')
                            job_post.append(try__3.text.strip())
                        except:
                            job_post.append('No_salary')
                    #add job summary
                    base_url = 'http://www.indeed.com'
                    for a in div.find_all(name = 'a',attrs = {'data-tn-element':'jobTitle'}):
                        c = base_url + str(a['href'])
                        print(c)
                        r = requests.get(c)
                        html_doc = r.text
                        soup = BeautifulSoup(html_doc,'html.parser')
                        summary = extract_job_summary(soup)
                        job_post.append(summary)                        
                    sample_df2.loc[num] = job_post
                    
    Job_info = sample_df2.append(sample_df)

    return(Job_info)

Call the function with the required parameters to create the technical job set

In [2]:
df_q2 = extract_job_info(ca_city_set,us_city_set,roles = q2_roles, max_result = 11)

Call the function with the required parameters to create the MBA job set

In [8]:
# df_q3 = extract_job_info(ca_city_set,us_city_set,roles = q3_roles, max_result = 11)

Run the following two files directly

In [9]:
df_q2 = pd.read_csv('df_q2.csv')

In [10]:
df_q3 = pd.read_csv('df_q3.csv')

Explore the job counts for the two categories

In [11]:
city_counts = pd.DataFrame(df_q2['city'].value_counts())
city_counts2 = pd.DataFrame(df_q3['city'].value_counts())

In [12]:
city_counts2

,city
San+francisco,476
New+york,248
Toronto,241
Boston,235
Chicago,221
Redmond,218
Seattle,213
Dallas,212
Vancouver,204
Raleigh,194


Now use the stopwords list provided for the assignment '1' to clean the summaries.

In [13]:
stop = open('stop_words.txt','r').read()
stop = stop.split('\n')
stop = set(stop)

In [14]:
df_q3['Summaries'] = df_q3['Summaries'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [15]:
# df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].replace('[!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~]', ' ', regex=True)
# df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].replace('[\\n\\n]+', ' ', regex=True)
# df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].replace('[\\n]+', ' ', regex=True)
# df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].str.lower()

In [16]:
# df_q3 = df_q3.drop(df_q3.columns[0], axis=1)

In [17]:
df_q3.head()

,city,job_title,company_name,location,salary,Summaries
0,San+francisco,"Analyst, Consulting",The Croner Company,"San Francisco Bay Area, CA",No_salary,"[""Our consulting practice expanding, experienc..."
1,San+francisco,Professional Services Consultant (Mortgage),"Alight, Inc.","San Francisco, CA",No_salary,['The Role\n\nIn position will integral member...
2,San+francisco,Lead Mobile Testing Consultant,Litmus7 Systems Consulting Ltd.,"San Francisco, CA 94102",No_salary,['Job Description\nExperience working fast pac...
3,San+francisco,Lead ATG testing consultant,Litmus7 Systems Consulting Ltd.,"San Francisco, CA 94102",No_salary,"['Must experience testing ATG, API, 3rd party ..."
4,San+francisco,People Consultant,Google,"San Bruno, CA",No_salary,['People Operations strives revolutionize huma...


## Now using Kaggle Data set for compiling a List of skillls presently Used in Job Market

Ultimately we created skills_algortihms and skill_tool from kaggle data set. Later we created a seperate list for languages we taught languages and tools should be seperate

IMporting the kaggle data set csv files

In [18]:
uf = pd.read_csv('multipleChoiceResponses.csv',encoding='latin-1')
fuf = pd.read_csv('freeformResponses.csv',encoding='latin-1')

C:\Users\Naga Raja Paidimarri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (31,83,86,87,98,99,109,116,123,124,127,129,130,164) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Naga Raja Paidimarri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (5,17,21,38,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
# uf.to_csv('kaggle_skills')
# fuf.to_csv('kaggle_skillsf')

In [20]:
# uf = pd.read_csv('kaggle_skills',encoding='latin-1')
# fuf = pd.read_csv('kaggle_skillsf',encoding='latin-1')

In [21]:
def split(x):
    x = x.dropna()
    x = x.tolist()
    x = ','.join(x)
    x = x.split(",")
    x = set(x)
    x = list(set(x))
    return x


Now I iported all Columns wheere can I get required skill in data set and combined them.

In [22]:
skills_ML_Tool= uf['MLToolNextYearSelect'] #t
skills_ML_Tool_free = fuf['MLToolNextYearFreeForm'] #t
skills_ML_Method= uf['MLMethodNextYearSelect'] #s
skills_ML_Method_free = fuf['MLMethodNextYearFreeForm'] #s
skills_lang= uf['LanguageRecommendationSelect'] #t
skills_lang_free = fuf['LanguageRecommendationFreeForm'] #t

Jobskills = ['Enterprise Tools','Visualizations', 'kaggle ranking', 'MOOC']  #s

Impactfulskill_free = fuf['ImpactfulAlgorithmFreeForm'] #s
 
skills_ML = uf['MLSkillsSelect'] #s
skills_ML_free = fuf['MLSkillsFreeForm'] #s
techniques_ML = uf['MLTechniquesSelect'] #s
techniques_ML_free = fuf['MLTechniquesFreeform'] #s

WorkLibrary_free = fuf['WorkLibrariesFreeForm'] #t

Work_skills_Algorithms = uf['WorkAlgorithmsSelect'] #s
Work_skills_Algorithms_free = fuf['WorkAlgorithmsFreeForm'] #s

Work_skills_tools = uf['WorkToolsSelect'] #t
Work_skills_tools_free1 = fuf['WorkToolsFreeForm1'] #t
Work_skills_tools_free2 = fuf['WorkToolsFreeForm2'] #t
Work_skills_tools_free3 = fuf['WorkToolsFreeForm3'] #t

WorkTools = ['AmazonML','Amazon Machine Learning','Amazon Web services','Angoss','C','C/C++','Cloudera','DataRobot','Flume',
             'GCP','Google Cloud Compute','Hadoop','Hive','pig','IBM Cognos','IBM SPSS Modeler','IBM SPSS Statistics',
             'IBM Watson','Impala','Java','Julia','Jupyter','KNIME','Mathematica','octave', 'MATLAB','Azure','Excel',
             'Microsoft R','Microsoft SQL','Minitab','NoSQL','Oracle','Orange','Perl','Python','Qlikview','R',
             'RapidMiner','Salfrod','SAP Business Objects','SAS Base','SAS Enterprise','SAS JMP','Spark','SQL',
             'Stan','Statistica','Tableau','TensorFlow','TIBCO','Unix'] #t


work_skills_method = uf['WorkMethodsSelect'] #s
work_skills_method_free1 = fuf['WorkMethodsFreeForm1'] #s
work_skills_method_free2 = fuf['WorkMethodsFreeForm2'] #s
work_skills_method_free3 = fuf['WorkMethodsFreeForm3'] #s

WorkMethod = ['A/B Testing','Association Rules','Bayesian Techniques','CNNs','Collaborative Filtering','Cross Validation',
              'Data Visualization','Decision Trees','Ensemble Methods','Evolutionary Approaches','GANs',
              'Gradient Boosted Machines','HMMs','kNN','Clustering','Lift Analysis','Logistic Regression','Markov Logic Networks'
              'Naive Bayes','Natural Language Processing','Neural Networks','PCA ','Dimensionality Reduction',
              'Prescriptive Modeling','Random Forests','Recommender Systems','RNNs','Segmentation','Simulation','SVMs',
              'Text Analytics','Time Series Analysis'] #s


In [23]:
# now grouping all skill related columns
skills_ML_Method = split(skills_ML_Method) #s
skills_ML_Method_free = split(skills_ML_Method_free) #s
# Jobskills = split(Jobskills) #s
Impactfulskill_free = split(Impactfulskill_free) #s
skills_ML = split(skills_ML) #s
skills_ML_free = split(skills_ML_free) #s
techniques_ML = split(techniques_ML) #s
techniques_ML_free = split(techniques_ML_free) #s
Work_skills_Algorithms = split(Work_skills_Algorithms) #s
Work_skills_Algorithms_free = split(Work_skills_Algorithms_free) #s
work_skills_method = split(work_skills_method) #s
work_skills_method_free1 = split(work_skills_method_free1) #s
work_skills_method_free2 = split(work_skills_method_free2) #s
work_skills_method_free3 = split(work_skills_method_free3) #s
# WorkMethod = split(WorkMethod)

# now grouping all tools related columns
skills_ML_Tool= split(skills_ML_Tool) #t
skills_ML_Tool_free = split(skills_ML_Tool_free) #t
skills_lang = split(skills_lang) #t
skills_lang_free = split(skills_lang_free) #t
WorkLibrary_free = split(WorkLibrary_free) #t
Work_skills_tools = split(Work_skills_tools) #t
Work_skills_tools_free1 = split(Work_skills_tools_free1) #t
Work_skills_tools_free2 = split(Work_skills_tools_free2) #t
Work_skills_tools_free3 = split(Work_skills_tools_free3) #t
# WorkTools = split(WorkTools)


In [24]:
skills_algorithms = skills_ML_Method + Jobskills + skills_ML +techniques_ML + Work_skills_Algorithms + work_skills_method + WorkMethod
skills_tools = skills_ML_Tool + Work_skills_tools  + WorkTools

##The following list is obtained form TFID vectorizer on JOb data, The code for TFID is in the same notebook but after two sections in PART 4

In [25]:
shortlist_tech_skills_usingTFID=['agile methodology','data analysis','data architecture','automation','business intelligence',
                            'cloud computing','consulting','data engineering','data management','data modeling','data base','data warehouse','decision making',
                                'deep learning','feature engineering','financial engineering','finance','identify data','information technology','large scale data',
                                 'machine learning','pattern recognition','ML pipeline','pipeline',
                                'predictive analytics','project management','quantitative','querying','relational database','research',
                                'data structure','troubleshooting','data visualization','visualization','warehousing','accounting', 'finance', 'marketing', 
'strategic management','structured data', 'unstructured data', 'digital marketing', 'product design']

In [26]:
shortlisted_skillsmgt=['vendor management','stakeholder management',
    'business administration','statistics','strategy management','strategy',
    'data insight','networking','problem solving','relationship management',
    'financial analysis','innovation','investment banking','managerial skills'
    'data driven','data analysis','database','decis making','economics','enterprise',
    'change management','project coordination', 'commerce','customer oriented','customer service',
    'business intelligence','business process','business requirements','business need',
    'agile project management','project management','architectures','asset management','automation']

Now I add both lists from kaggle data set and also job descritptions. we use the combined list for futher analysis of second and third question

In [27]:
skills_algorithms_tech = skills_algorithms + shortlist_tech_skills_usingTFID
skills_algorithms_mgmt = skills_algorithms + shortlisted_skillsmgt

The soft skills is form TFID Part 4

In [28]:
softskills=['time management','verbal communication','written communication',
    'motivated','passionate','problem solving','report making','reporting','responsible','strategic','team work',
    'fast paced','focused','goal oriented','group work','independent','initiative','innovative','interpersonal skill','leadership','project management'
    'data driven','creative','agile','challenging','client management','collaborative','commitment','communication skill','competitive','proactive','written communication','verbal communication',
    'initiative','interpersonal skills','independent','methodological','organized','passionate',
    'deadline oriented','detail oriented','self driven','dynamic','firm','fast paced'
    'creative','client management','challenging','collaborative','commitment','communication skill','competitive',
    'action oriented','agile','analytical','attention to detail']

In [29]:
def lower(x):
    x = list(set(x))
    for i in range(0,len(x)):
        x[i] = x[i].lower()
        
    return x

In [30]:
stop = open('stop_words.txt','r').read()
stop = stop.split('\n')
stop = set(stop)
stop = list(stop) + list(string.ascii_lowercase) + lower(skills_lang)
stop = set(stop)

In [31]:
def stopwords(x):
    for i in range(0,len(x)):
        x[i] = ' '.join([word for word in x[i].split() if word not in (stop)])
    return x


In [32]:
def removebiglines(x):
    for i in range(0,len(x)):
        if len(x[i].split()) > 7:
            x[i] = ''
    return x

In [33]:
def remove(x):
    list_of_words = ['other',"don't","no","x80"]
    for j in range (0,len(list_of_words)):
        for i in range (0,len(x)):
            if x[i].find(list_of_words[j]) != -1:
                x[i] = ''
    x = list(set(x))
    return x

In [34]:
def clean(x):
    x = ','.join(x)
    x = x.replace('-',',')
    x = x.replace(')',',')
    x = x.replace('(',',')
    x = x.replace('e.g.',',')
    x = x.replace(';',',')
    x = x.replace('/',',')
    x = x.split(",")
    x = list(set(x))
    x = list(filter(None, x))
    for i in range (0,len(x)):
        x[i] = x[i].strip()
        x[i] = x[i].rstrip('s')
    x = list(set(x))

#     df['Words'] = np.where(df['Words'].str[-1] == 's', df['Words'].str[:-1], df['Words'])
    return x

In [35]:
def count(x,y):
    c = []
    for i in range(0,len(y)):
        c.append(0)      
    for i in range(0,len(y)):
        for j in range(0,len(x)):
            c[i] = c[i] + x.iloc[j,5].count(y[i])
    return c      

<p style="color:black;font-family:times new roman;font-size:140%;"> count_once function for the frequnecy count<p/>

In [36]:
def count_once(x,y):
    c = []
#     d = pd.DataFrame(0, index=np.arange(len(data)), columns=feature_list)
    global jobs    
    jobs = pd.DataFrame(0,index=np.arange(len(x)),columns = y)
    for i in range(0,len(y)):
        c.append(0)      
    for i in range(0,len(y)):
        for j in range(0,len(x)):
            if (x.iloc[j,5].count(y[i]) > 0):
                jobs.iloc[j,i] = 1
                c[i] = c[i] + 1
                
    return c

In [37]:
def frequency(x,y):
    nb = int(input('write "1" for count function and "2" for count_once function: '))
    if nb == 1:
        f = count(x,y)
    if nb == 2:
        f = count_once(x,y)
    return f


In [38]:
def printfreq(x,y):
#     f = count(x,y)
    for i in range (0,len(y)):
        if x[i] > 0:
            print(y[i],x[i])

In [39]:
def final(y):
    y = stopwords(y)
    y = removebiglines(y)
    y = lower(y)
    y = remove(y)
    y = clean(y)
    y = stopwords(y)
    y = clean(y)
    return y
    

## The following are the final list of algorithms for tech and managment

In [40]:
algorithms_tech = final(skills_algorithms_tech)

In [41]:
algorithms_mgmt = final(skills_algorithms_mgmt)

In [42]:
y2 = algorithms_mgmt
y2.extend(['mars','markov logic network','naive baye','evolutionary algorithm','genetic algorithm','dimensionality reduction','hidden markov model','enterprise tool','svm','gan','a/b testing','cross validation'])
y2.remove('testing')
y2.remove('cro')
y2.remove('validation')
y2.remove('ds method')
y2.remove('plan learning')
y2.remove('mar')
y2 = list(set(y2))
algorithms_mgmt = y2

<p style="color:black;font-family:times new roman;font-size:120%;"> list for algorithms for management job postings<p/>

In [43]:
algorithms_mgmt

['business requirement',
 'outlier detection',
 'decis making',
 'investment banking',
 'unsupervised learning',
 'collaborative filtering',
 'change management',
 'machine translation',
 'genetic algorithm',
 'statistic',
 'business intelligence',
 'asset management',
 'busine',
 'commerce',
 'supervised machine learning',
 'gradient boosting',
 'deep learning',
 'relationship management',
 'factor analysi',
 'lift analysi',
 'pca',
 'bayesian technique',
 'association rule',
 'link analysi',
 'boosting',
 'simulation',
 'problem solving',
 'automation',
 'text mining',
 'hidden markov models hmm',
 'markov logic network',
 'business administration',
 'time serie',
 'networking',
 'computer vision',
 'text analytic',
 'naive baye',
 'cross validation',
 'genetic & evolutionary algorithm',
 'recommender system',
 'managerial skillsdata driven',
 'enterprise',
 'agile project management',
 'prescriptive modeling',
 'cluster analysi',
 'time series analysi',
 'bayesian method',
 'random 

In [44]:
# skills_algorithms

In [45]:
# y2

In [46]:
y2 = algorithms_tech
y2.extend(['mars','markov logic network','naive baye','evolutionary algorithm','genetic algorithm','dimensionality reduction','hidden markov model','enterprise tool','svm','gan','a/b testing','cross validation'])
y2.remove('testing')
y2.remove('cro')
y2.remove('validation')
y2.remove('ds method')
y2.remove('plan learning')
y2.remove('mar')
y2 = list(set(y2))
algorithms_tech = y2

<p style="color:black;font-family:times new roman;font-size:120%;"> list for algorithms for technical job postings<p/>

In [47]:
algorithms_tech

['data modeling',
 'agile methodology',
 'accounting',
 'outlier detection',
 'querying',
 'strategic management',
 'digital marketing',
 'unsupervised learning',
 'collaborative filtering',
 'decision making',
 'machine translation',
 'genetic algorithm',
 'large scale data',
 'business intelligence',
 'consulting',
 'data engineering',
 'supervised machine learning',
 'deep learning',
 'gradient boosting',
 'factor analysi',
 'lift analysi',
 'pca',
 'cloud computing',
 'bayesian technique',
 'association rule',
 'link analysi',
 'boosting',
 'simulation',
 'automation',
 'text mining',
 'hidden markov models hmm',
 'markov logic network',
 'pipeline',
 'time serie',
 'computer vision',
 'text analytic',
 'data management',
 'naive baye',
 'cross validation',
 'genetic & evolutionary algorithm',
 'recommender system',
 'prescriptive modeling',
 'cluster analysi',
 'time series analysi',
 'feature engineering',
 'bayesian method',
 'random forest',
 'speech recognition',
 'rule induct

In [48]:
# skills_algorithms_dataframe = pd.read_csv('rel_frequencies.csv')

In [49]:
# skills_algorithms_2 = list(skills_algorithms_dataframe['list'])

<p style="color:black;font-family:times new roman;font-size:120%;"> list for tools for job postings<p/>

In [50]:
tools = final(skills_tools)

In [51]:
tools .remove('c++')
tools .remove('mar')
tools .extend(['mars'])

In [52]:
tools 

['rapidminer',
 'statsoft',
 'tibco',
 'flume',
 'commercial version',
 'jupyter',
 'google cloud compute',
 'hive',
 'rf',
 'weka',
 'mllib',
 'salfrod systems cart',
 'microsoft excel data mining',
 'oracle',
 'stan',
 'statistica',
 'spark',
 'microsoft server',
 'microsoft',
 'amazon machine learning',
 'revolution analytic',
 'perl',
 'sap businessobjects predictive analytic',
 'sap business object',
 'salfrod',
 'jmp',
 'amazonml',
 'tensorflow',
 'enterprise miner',
 'cloudera',
 'ibm spss modeler',
 'tibco spotfire',
 'ango',
 'impala',
 'pig',
 'gcp',
 'unix',
 'quest',
 'microsoft azure machine learning',
 'treenet',
 'free version',
 'unix shell',
 'tableau',
 'oracle data mining',
 'waton analytic',
 'minitab',
 'octave',
 'awk',
 'microsoft server data mining',
 'amazon web service',
 'oracle enterprise',
 'ibm watson',
 'dell',
 'enterprise',
 'orange',
 'excel',
 'knime',
 'base',
 'ibm spss statistic',
 'hadoop',
 'azure',
 'spm',
 'mathematica',
 'datarobot',
 'qlikvie

<p style="color:black;font-family:times new roman;font-size:120%;"> list for programming languages for job postings<p/>

In [53]:
languages = lower(skills_lang)

In [54]:
languages.extend(['c++','c#'])
#languages.remove('r')

In [55]:
languages

['sas',
 'matlab',
 'julia',
 'other',
 'sql',
 'java',
 'haskell',
 'python',
 'scala',
 'f#',
 'c/c++/c#',
 'stata',
 'r',
 'c++',
 'c#']

<p style="color:black;font-family:times new roman;font-size:120%;"> list for soft skills for job postings<p/>

In [56]:
softskills = final(softskills)

In [57]:
softskills

['verbal communication',
 'problem solving',
 'project managementdata driven',
 'collaborative',
 'focused',
 'interpersonal skill',
 'detail oriented',
 'dynamic',
 'independent',
 'fast pacedcreative',
 'client management',
 'group work',
 'firm',
 'initiative',
 'attention detail',
 'written communication',
 'competitive',
 'organized',
 'agile',
 'deadline oriented',
 'proactive',
 'passionate',
 'responsible',
 'action oriented',
 'commitment',
 'analytical',
 'methodological',
 'strategic',
 'report making',
 'challenging',
 'communication skill',
 'fast paced',
 'goal oriented',
 'reporting',
 'leadership',
 'driven',
 'team work',
 'motivated',
 'creative',
 'time management']

In [58]:
def dict(x,y):
    dictionary = dict(zip(x,y))
    return dictionary

<p style="color:black;font-family:times new roman;font-size:120%;"> the following functions obtain the frequency count for r, c++, c because they cannot be obtained via the previous count_once function<p/>

In [59]:
def _freq_of_r(text):
    r_lang =[]
    for x in str(text).split():
        Temp = re.findall(r'^r$',str(x))
        if Temp != []:
            r_lang.append(Temp)
    return len(r_lang)

def _freq_of_cplusplus(text):
    r_lang =[]
    for x in str(text).split():
        Temp = re.findall(r'^c\+\+$', x)
        if Temp != []:
            r_lang.append(Temp)
    return len(r_lang)

def _freq_of_c(text):
    r_lang =[]
    for x in str(text).split():
        Temp = re.findall(r'^c$', x)
        if Temp != []:
            r_lang.append(Temp)
    return len(r_lang)

In [60]:
df_q2

,city,job_title,company_name,location,salary,Summaries
0,San+francisco,Clinical Data Analyst,Meritage Medical Network,"San Francisco Bay Area, CA",No_salary,"[""Who We AreMeritage Medical Network (MMN) is ..."
1,San+francisco,Data Scientist,Walmart eCommerce,"Sunnyvale, CA",No_salary,['Position Description\n\nYou will have the op...
2,San+francisco,Junior Data Analyst,Realogy Corporate,"Emeryville, CA",No_salary,['Requisition 51649\n\nAbout ZapLabs LLC\nZapL...
3,San+francisco,Data Manager,Need to Impeach,"San Francisco, CA",No_salary,['Need To Impeach is seeking a data and analyt...
4,San+francisco,Data Analyst,Brandless,"San Francisco, CA",No_salary,"[""Brandless is reinventing modern consumption ..."
5,San+francisco,Data Science Intern,Square,"San Francisco, CA 94103 (South Of Market area)",No_salary,['Job Description\n\nYou will:\nPartner with S...
6,San+francisco,Business Analyst (Entry-Level Business Consult...,Nimble Consulting Inc.,"San Francisco, CA",No_salary,['Business Analyst\n\n(This is not a technical...
7,San+francisco,Data Analyst I,Bracket Global,"San Francisco, CA 94107 (South Of Market area)",No_salary,['The Data Analyst I is responsible for workin...
8,San+francisco,Data Analyst,Nima,"San Francisco, CA",No_salary,"['At the intersection of hardware, software an..."
9,San+francisco,Data Analyst - Marketing Analytics,Tripping.com,"San Francisco, CA 94105 (Financial District area)",No_salary,['Job Description\n\nThe Challenge\n\nOur prod...


## Part 3: Basic Analysis using the Lists

## The main aim of this part is to create a 'score' for each skill which means we want to know how many percent of jobs a skill appears. This term is relative frequency which is simply percent on jobs it appeared

## The result of this part are to two csv files name 'Mgmt_relative_frequency' and 'Technical_relative_frequency'. These are submitted along with report. These two csv files are used in further notebooks

In [61]:
#applying the count_once function on the technical and management dataframes
newxx=count_once(df_q2,algorithms_tech)
newyy=count_once(df_q3,algorithms_mgmt)

In [62]:
finale=pd.DataFrame({'list':algorithms_tech})
finale['technical']=finale.apply(lambda _:'', axis=1)
finale['technical']=newxx

In [63]:
robot=pd.DataFrame({'list':algorithms_mgmt})
robot['management']=robot.apply(lambda _:'', axis=1)
robot['management']=newyy

In [64]:
#function to calculate relative frequency

def relative(df):
    cabe=[]

    for row in df:
        row =row/df.sum()
        cabe.append(row)
    return cabe


In [65]:
#applying the relative function on the frequency to produce relative frequency
arsenal=relative(finale['technical'])
spurs=relative(robot['management'])

In [66]:
finale['technical rf']=finale.apply(lambda _:'', axis=1)
robot['management rf']=robot.apply(lambda _:'', axis=1)
finale['technical rf']=arsenal
robot['management rf']=spurs

# finale.to_csv('finale1.csv')
# robot.to_csv('robot.csv')

In [67]:
barca=count_once(df_q2,tools)
madrid=count_once(df_q2,languages)
atletico=count_once(df_q2,softskills)

In [68]:
inter=count_once(df_q3,tools)
lazio=count_once(df_q3,languages)
milan=count_once(df_q3,softskills)

In [69]:
df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].replace('[!"$%&\'()*,-./:;<=>?[\\]^_`{|}~]', ' ', regex=True)
df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].replace('[\\n\\n]', ' ', regex=True)
#df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].replahttps://labs.cognitiveclass.ai/bdu/logince('[\\n]+', ' ', regex=True)
df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].str.lower()

In [70]:
df_q3.loc[:, "Summaries"] = df_q3.loc[:, "Summaries"].replace('[!"$%&\'()*,-./:;<=>?[\\]^_`{|}~]', ' ', regex=True)
df_q3.loc[:, "Summaries"] = df_q3.loc[:, "Summaries"].replace('[\\n\\n]', ' ', regex=True)
#df_q2.loc[:, "Summaries"] = df_q2.loc[:, "Summaries"].replahttps://labs.cognitiveclass.ai/bdu/logince('[\\n]+', ' ', regex=True)
df_q3.loc[:, "Summaries"] = df_q3.loc[:, "Summaries"].str.lower()

In [71]:
siroc=list(df_q2['Summaries'])
saints=_freq_of_r(siroc)

In [72]:
brom=_freq_of_cplusplus(siroc)

In [73]:
swans=_freq_of_c(siroc)

In [74]:
samy=list(df_q3['Summaries'])
rockets=_freq_of_r(samy)

In [75]:
okc=_freq_of_cplusplus(samy)

In [76]:
parker=_freq_of_c(samy)

In [77]:
serie_a=pd.DataFrame({'list':tools})
serie_a['technical']=serie_a.apply(lambda _:'', axis=1)
serie_a['technical']=barca
# serie_a.to_csv('serie_a.csv')

In [78]:
bayern=pd.DataFrame({'list':tools})
bayern['management']=bayern.apply(lambda _:'', axis=1)
bayern['management']=inter
# bayern.to_csv('bayern.csv')

In [79]:
gunners=pd.DataFrame({'list':languages})
gunners['technical']=gunners.apply(lambda _:'', axis=1)
gunners['technical']=madrid
# gunners.to_csv('gunners.csv')

In [80]:
liverpoo=pd.DataFrame({'list':languages})
liverpoo['management']=liverpoo.apply(lambda _:'', axis=1)
liverpoo['management']=lazio
# liverpoo.to_csv('liverpool.csv')

In [81]:
gunners.iloc[2,1]=saints
gunners.iloc[13,1]=brom
gunners.iloc[12,1]=swans

In [82]:
liverpoo.iloc[2,1]=rockets
liverpoo.iloc[13,1]=parker
liverpoo.iloc[12,1]=okc

In [83]:
shakhter=pd.DataFrame({'list':softskills})
shakhter['technical']=shakhter.apply(lambda _:'', axis=1)
shakhter['technical']=atletico
# shakhter.to_csv('shakhter.csv')

In [84]:
lyon=pd.DataFrame({'list':softskills})
lyon['management']=lyon.apply(lambda _:'', axis=1)
lyon['management']=milan
# lyon.to_csv('lyon.csv')

In [85]:
finales=pd.DataFrame({'list':algorithms_tech})
finales['technical']=finales.apply(lambda _:'', axis=1)
finales['technical']=newxx
# finales.to_csv('finales.csv')

In [86]:
semis=pd.DataFrame({'list':algorithms_mgmt})
semis['management']=semis.apply(lambda _:'', axis=1)
semis['management']=newyy
# semis.to_csv('semis.csv')

## Part 4: Using TFID vectorizer to create a list of algorithms, tools and languages from Indeed Job postings obtained from part 1

##Now we also obtained in demand skills not only from kaggle data but also from our own job discriptions

In [87]:
import pandas as pd
import nltk
import numpy as np
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
import re

In [88]:
#Reading Web scraped tech data analyst jobs

data=pd.read_csv('df_q3.csv', encoding='utf8')
data

,city,job_title,company_name,location,salary,Summaries
0,San+francisco,"Analyst, Consulting",The Croner Company,"San Francisco Bay Area, CA",No_salary,"[""Our consulting practice is expanding, and we..."
1,San+francisco,Professional Services Consultant (Mortgage),"Alight, Inc.","San Francisco, CA",No_salary,['The Role\n\nIn this position you will be an ...
2,San+francisco,Lead Mobile Testing Consultant,Litmus7 Systems Consulting Ltd.,"San Francisco, CA 94102",No_salary,['Job Description\nExperience working in a fas...
3,San+francisco,Lead ATG testing consultant,Litmus7 Systems Consulting Ltd.,"San Francisco, CA 94102",No_salary,"['Must have experience in testing ATG, API, 3r..."
4,San+francisco,People Consultant,Google,"San Bruno, CA",No_salary,['People Operations strives to revolutionize h...
5,San+francisco,Entry Level Business Analyst,American CodeForce,"San Francisco, CA",No_salary,"[""Entry-level Business Analysts work under sup..."
6,San+francisco,Business Strategy Senior Associate,Slack,"San Francisco, CA",No_salary,"['Our team works on global, cross-functional p..."
7,San+francisco,Mobile Strategist,Apptimize,"San Francisco, CA",No_salary,"[""ABOUT THE ROLE\n\nLooking for the combinatio..."
8,San+francisco,ParthenonEY - Life Sciences - Consultant- San ...,EY,"San Francisco, CA",No_salary,"["". Competitive business today is all about ma..."
9,San+francisco,Consultant,"Motive Power, Inc.","San Francisco, CA 94105 (Financial District area)",No_salary,"[""Job SummaryProject Management Consultant Res..."


In [89]:
#Extracting job summaries from the dataset for mining most in demand skills

summaries=data['Summaries'].tolist()

In [90]:
#strips html formatting and converts to unicode

summaries_html_clean = []
for text in summaries:
    text = BeautifulSoup(text, 'html.parser').getText()
    summaries_html_clean.append(text)
summaries=summaries_html_clean
summaries

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [91]:
# load nltk's English stopwords as variable called 'stopwords'
# use nltk.download() to install the corpus first
# Stop Words are words which do not contain important significance to be used in Search Queries
stopwords = nltk.corpus.stopwords.words('english')

In [92]:
#Text cleaning !

corpus = [] # defining an empty set in which we will add cleansed data


for i in range(len(summaries)):
    review = re.sub('[\s]+', " ", summaries[i]) #removing whitespaces
    review = re.sub(r'[a-z]*[:.]+\S+', " ", review) #Removing non http urls
    review = re.sub(r'\\n', " ", review) #Remove newline tags
    review= re.sub('[^a-zA-Z0-9]+?',' ',review) # removing special characters
    review= review.lower() #converting all words to lower case
    review= review.split()
    ps = PorterStemmer()
    review= [ps.stem (word) for word in review]    #Stemming words
    review=[word for word in review if not word in stopwords]   #removing stop word
    review = '  '.join(review)
    corpus.append(review)


len(corpus)
summaries_clean=corpus


In [93]:
summaries_clean[1]

'role  thi  posit  integr  member  project  team  work  close  senior  member  firm  throughout  phase  engag  consult  engag  custom  configur  alight  mortgag  lend  applic  cloud  base  enterpris  plan  solut  identifi  understand  client  need  guid  process  implement  train  applic  success  applic  abl  identifi  understand  compani  financi  plan  need  must  work  well  team  environ  resourc  independ  self  strong  team  player  posit  servic  orient  attitud  startup  mindset  anyth  everyth  self  starter  great  work  ethic  analyt  thinker  superior  problem  solv  decis  make  skill  possess  initi  creat  present  analysi  scratch  answer  question  pose  look  detail  interpret  impact  key  busi  driver  extrem  detail  orient  organ  excel  multi  tasker  ha  strong  time  project  manag  skill  understand  import  plan  advanc  meet  deadlin  includ  multipl  review  point  iter  strong  commun  written  verbal  reliabl  respons  email  phone  b  requir  mba  stron

In [94]:
#Defining function to tokenize text for TF-IDF

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens



max_df: this is the maximum frequency within the documents a given feature can have to be used in the tfi-idf matrix. If the term is in greater than 80% of the documents it probably cares little meanining

min_idf: this could be an integer (e.g. 5) and the term would have to be in at least 5 of the documents to be considered. Here I pass 0.2; the term must be in at least 20% of the document.

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=20000,
                                 min_df=0.1, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(summaries_clean) #fit the vectorizer to summaries_clean

# (2649, 563) means the matrix has 100 rows and 532 columns
print(tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()
len(terms)

Wall time: 9.01 s
(2649, 528)


528

In [96]:
shortlisted_skillsmgt=['vendor management','stakeholder management','driver license','web technology',
    'management','business administration','statistics','strategy management','strategy',
    'data insight','networking','problem solving','relationship management',
    'finance','financial analysis','innovation','investment banking','managerial skills'
    'dashboard','data driven','data analysis','database','decis making','economics','enterprise',
    'change management','project coordination', 'commerce','complex project','critical','customer oriented','customer service',
    'business intelligence','big data','business analyst','business process','business requirements','business need',
    'agile project management','project management','anlytical skill','architectures','asset management','automation']

shortlist_skills_soft=['proactive','written communication','verbal communication',
    'initiative','interpersonal skills','independent','methodological','organized','passionate',
    'deadline oriented','detail oriented','self driven','dynamic','firm','fast paced'
    'creative','client management','challenging','collaborative','commitment','communication skill','competitive',
    'action oriented','agile','analytical','attention to detail']

degrees_preferred=['bachelor','computer science','master','master of business administration','software']

shortlist_toolsmgt=['c','microsoft office','excel','powerpoint','presentation','r','sql','tableau']

### Now repeating the same TFID procedure above for the techncial jobs

In [97]:
import pandas as pd
import nltk
import numpy as np
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
import re

### <p style="color:black;font-family:times new roman;font-size:120%;"> Get required data </p>
<p style="color:black;font-family:times new roman;font-size:140%;"> A dataset of data analyst job was created using web scraping of indeed website. This data is extracted into a pandas data frame. The data is then cleaned by case conversion and stemming. </p>


In [107]:
data=pd.read_csv('df_q2.csv', encoding='utf8')

In [108]:
data

,city,job_title,company_name,location,salary,Summaries
0,San+francisco,Clinical Data Analyst,Meritage Medical Network,"San Francisco Bay Area, CA",No_salary,"[""Who We AreMeritage Medical Network (MMN) is ..."
1,San+francisco,Data Scientist,Walmart eCommerce,"Sunnyvale, CA",No_salary,['Position Description\n\nYou will have the op...
2,San+francisco,Junior Data Analyst,Realogy Corporate,"Emeryville, CA",No_salary,['Requisition 51649\n\nAbout ZapLabs LLC\nZapL...
3,San+francisco,Data Manager,Need to Impeach,"San Francisco, CA",No_salary,['Need To Impeach is seeking a data and analyt...
4,San+francisco,Data Analyst,Brandless,"San Francisco, CA",No_salary,"[""Brandless is reinventing modern consumption ..."
5,San+francisco,Data Science Intern,Square,"San Francisco, CA 94103 (South Of Market area)",No_salary,['Job Description\n\nYou will:\nPartner with S...
6,San+francisco,Business Analyst (Entry-Level Business Consult...,Nimble Consulting Inc.,"San Francisco, CA",No_salary,['Business Analyst\n\n(This is not a technical...
7,San+francisco,Data Analyst I,Bracket Global,"San Francisco, CA 94107 (South Of Market area)",No_salary,['The Data Analyst I is responsible for workin...
8,San+francisco,Data Analyst,Nima,"San Francisco, CA",No_salary,"['At the intersection of hardware, software an..."
9,San+francisco,Data Analyst - Marketing Analytics,Tripping.com,"San Francisco, CA 94105 (Financial District area)",No_salary,['Job Description\n\nThe Challenge\n\nOur prod...


##### Creating list of job description which we will clean and do vectorizing using TF IDF

In [109]:
summaries=data['Summaries'].tolist()

In [110]:
len(summaries)

3190

In [113]:
#strips html formatting and converts to unicode
summaries_html_clean = []
for text in summaries:
    text = BeautifulSoup(text, 'html.parser').getText()
    summaries_html_clean.append(text)
summaries=summaries_html_clean
summaries

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [114]:
# nltk.download()

In [115]:
# load nltk's English stopwords as variable called 'stopwords'
# use nltk.download() to install the corpus first
# Stop Words are words which do not contain important significance to be used in Search Queries
stopwords = nltk.corpus.stopwords.words('english')

"""# load nltk's SnowballStemmer as variabled 'stemmer'
stemmer = SnowballStemmer("english")"""

'# load nltk\'s SnowballStemmer as variabled \'stemmer\'\nstemmer = SnowballStemmer("english")'

In [116]:
len(stopwords)

179

#### Performing text cleaning on job description data

In [117]:
#Text cleaning !

corpus = [] # defining an empty set in which we will add cleansed data


for i in range(len(summaries)):
    #review = re.sub(r'^https?:\/\/.*[\r\n]*', '\t', summaries[i], flags=re.MULTILINE) #removing all https://
    review = re.sub('[\s]+', " ", summaries[i]) #removing whitespaces
    review = re.sub(r'[a-z]*[:.]+\S+', " ", summaries[i]) #Removing non http urls
    review= re.sub('[^a-zA-Z0-9]+?',' ',review) # removing special characters
    review= review.lower() #converting all words to lower case
    review= review.split()
    ps = PorterStemmer()
    review= [ps.stem (word) for word in review]    #Stemming words
    review=[word for word in review if not word in stopwords]   #removing stop word
    review = '  '.join(review)
    corpus.append(review)


len(corpus)
summaries_clean=corpus

Creating tokenizer function

In [118]:

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens



### Using TF IDF vectorizer to extract skills from job description

For information retrieval, tf–idf or TFIDF, short for 'term frequency–inverse document frequency', is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. 

It is often used as a weighting factor in searches of information retrieval, and text mining. The tf-idf value increases proportionally to the number of times a word appears in the document (in this case it could be skills, qualification, employment criteria, education background, degree requirements, terms and conditions etc.) and is offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general. TF-IDF is one of the most popular term-weighting schemes.

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=20000,
                                 min_df=0.3, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(summaries_clean) #fit the vectorizer to synopses

# (100, 563) means the matrix has 100 rows and 563 columns
print(tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()
len(terms)

Wall time: 9.97 s
(3190, 109)


109

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=20000,
                                 min_df=0.1, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(summaries_clean) #fit the vectorizer to synopses

# (100, 563) means the matrix has 100 rows and 563 columns
print(tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()
len(terms)

Wall time: 9.75 s
(3190, 494)


494

In [122]:
terms

['abil',
 'abl',
 'access',
 'account',
 'achiev',
 'action',
 'activ',
 'ad',
 'addit',
 'administr',
 'advanc',
 'age',
 'agil',
 'algorithm',
 'analys',
 'analysi',
 'analyst',
 'analyt',
 'analyz',
 'ani',
 'appli',
 'applic',
 'approach',
 'appropri',
 'architect',
 'architectur',
 'area',
 'assign',
 'assist',
 'associ',
 'attent',
 'autom',
 'avail',
 'aw',
 'bachelor',
 'background',
 'base',
 'benefit',
 'best',
 'best practic',
 'better',
 'bi',
 'big',
 'big data',
 'bring',
 'build',
 'busi',
 'busi intellig',
 'c',
 'candid',
 'capabl',
 'care',
 'career',
 'center',
 'challeng',
 'chang',
 'class',
 'client',
 'close',
 'cloud',
 'code',
 'collabor',
 'collect',
 'color',
 'color religion',
 'combin',
 'come',
 'commit',
 'commun',
 'commun skill',
 'compani',
 'competit',
 'complet',
 'complex',
 'comput',
 'comput scienc',
 'concept',
 'configur',
 'connect',
 'consid',
 'consist',
 'consult',
 'consum',
 'continu',
 'contribut',
 'control',
 'core',
 'corpor',
 'creat'

In [123]:
shortlist_tech_skills_usingTFID=['algorithm','agile methodology','analytic','data analysis','data architecture','automation','big data','business intelligence',
                            'cloud computing','skills','computer science','consulting','data engineering','data management','data modeling','database','data warehouse','decision making',
                                'deep learning','feature engineering','financial engineering','finance','identify data','information technology','large scale data',
                                 'machine learning','mathematics','quant','modeling','optimization','pattern recognition','ML pipeline','pipeline',
                                'predictive analytics','programming','coding','project management','quantitative','querying','relational database','research',
                                'data structure','troubleshooting','visualization','warehousing']

degree_preferred=['bachelor','engineering background','software engineering', 'software development','softwares','statistics', 'data storage']

shortlisled_softskills=['time management','verbal communication','written communication',''
    'motivated','passionate','problem solving','report making','reporting','responsible','strategic','team work',
    'fast paced','focused','goal oriented','group work','independent','initiative','innovative','interpersonal skill','leadership','project management'
    'data driven','creative','agile','challenging','client management','collaborative','commitment','communication skill','competitive','']

shortlisted_tools=['tableau',
    'c','cloud','azure','excel','hadoop','hive','java','linux','microsoft office','oracle','python','r','spark','sql','sql server','stack']

###1st Question Data

## Question 1 

## To answer we used the combination of tech skills obtained from kaggle data set and TFID vectorizer, the list is "algorithms_tech". and we compiled a list of introduction courses from diffferent universities in the file'1st Question Data'. After going through this file we understood on average each intro course is teaching around 70 algorithms. so we picked '70' topics from our combined technical skills list and used that to desing our intro course. And this intro course is a mandotory core course for both of our circullums

In [124]:
algorithms_tech

['machine translation',
 'structured data',
 'pca dimensionality reduction',
 'survival analysi',
 'lift analysi',
 'data structure',
 'rule induction',
 'predictive analytic',
 'feature engineering',
 'finance',
 'markov logic networksnaive baye',
 'data architecture',
 'reinforcement learning',
 'cnn',
 'pattern recognition',
 'cloud computing',
 'support vector machine',
 'natural language processing',
 'text analytic',
 'time series analysi',
 'marketing',
 'strategic management',
 'kaggle ranking',
 'data management',
 'enterprise tool',
 'simulation',
 'svm',
 'pca',
 'social network analysi',
 'hidden markov models hmm',
 'evolutionary approache',
 'data modeling',
 'association rule',
 'recommendation engine',
 'logistic regression',
 'naive baye',
 'adversarial learning',
 'cross validation',
 'bayesian method',
 'data warehouse',
 'machine learning',
 'business intelligence',
 'bayesian technique',
 'genetic & evolutionary algorithm',
 'unstructured data',
 'digital marketing